In [1]:
from utils import *
from utils2 import *
from gpcrdb_soup import *

In [2]:
import Bio
import yaml
import os
from functools import partial
from operator import is_not
from tqdm import tqdm

In [3]:
def disp3(df: pd.DataFrame):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df.head(3))

In [4]:
uniprot = False  # specify if we want to scrape for uniprot links ~

In [5]:
# table = get_table(reload=True, save=True, uniprot=uniprot)

In [6]:
table = get_table(reload=False)

In [7]:
if uniprot:
    t = table[['PDB', 'pdb_link', 'uniprot(gene)','uniprot_id', 'uniprot_link']]
else:
    t = table[['PDB', 'pdb_link', 'uniprot(gene)']]

In [8]:
# t
# Todo: https://www.ebi.ac.uk/training/online/sites/ebi.ac.uk.training.online/files/UniProt_programmatically_py3.pdf

In [9]:
df = table

# This is the thing we might want to specify with argparse
filtered = df[df['Cl.'].str.contains('A')]
filtered = filtered[filtered['Species'].str.contains('Human')]

In [10]:
# filtered

In [11]:
# 2.1) find active state (complex i.p.)
filtered_complex = filtered[filtered['Family'].str.contains('Gs')]
active = filtered_complex[filtered_complex['State'].str.contains('Active')]

In [12]:
# disp3(active)

In [13]:
# todo: check if file already exists and use overwrite=True/False
"""
for url in tqdm(active['pdb_link']):
    download_pdb(url, 'active')
"""

"\nfor url in tqdm(active['pdb_link']):\n    download_pdb(url, 'active')\n"

In [14]:
# Data loading
files_a, prots_a = get_pdb_files(path='data/pdb/active/')  # Get all downloaded pdb files in specified path
print("Found {} proteins: {}.".format(len(prots_a), prots_a))

Found 13 proteins: ['3SN6', '7CFN', '6E67', '7CFM', '7BZ2', '7DHI', '5G53', '6NI3', '7DHR', '7D7M', '6LI3', '6GDG', '7BW0'].


In [15]:
# find inactive counter parts of the active proteins ==> use gene/uniprot, Family and species to match them

genes = list(set(active['uniprot(gene)'].values.tolist()))
print(genes)

inactive = filtered[(filtered['uniprot(gene)'].isin(genes))
                    & (filtered['State'].str.contains('Inactive')) 
                    & (filtered['Species'].str.contains('Human'))]

['GPBAR', 'GPR52', 'ADRB2', 'AA2AR', 'PE2R4']


In [16]:
"""for url in tqdm(inactive['pdb_link']):
    download_pdb(url, 'inactive')"""

"for url in tqdm(inactive['pdb_link']):\n    download_pdb(url, 'inactive')"

In [17]:
# disp3(inactive)

In [18]:
# Data loading
files_i, prots_i = get_pdb_files(path='data/pdb/inactive/')  # Get all downloaded pdb files in specified path
print("Found {} proteins: {}.".format(len(prots_i), prots_i))

Found 74 proteins: ['5UIG', '6PRZ', '3EML', '5IU4', '2RH1', '3UZC', '3REY', '5OLV', '6LPJ', '5MZJ', '5OLO', '5N2R', '6OBA', '3PDS', '5D5A', '5JTB', '5YHL', '5OM4', '5MZP', '5OM1', '3VG9', '5OLZ', '5OLG', '5UVI', '6PS5', '5JQH', '2R4S', '3RFM', '5K2C', '6MH8', '5X7D', '5D6L', '3NYA', '5IU8', '5NM2', '6AQF', '6LPL', '3NY9', '6GT3', '6PS3', '6PS7', '3UZA', '2R4R', '5VRA', '6PS1', '4EIY', '6LPK', '5D5B', '5IUA', '5K2D', '3NY8', '3VGA', '5YWY', '6PS4', '6ZDR', '6WQA', '4GBR', '6S0Q', '6PS6', '3D4S', '5K2B', '6JZH', '6ZDV', '3PWH', '3KJ6', '6S0L', '5IUB', '5NM4', '5K2A', '5NLX', '6PS2', '5OLH', '5IU7', '6PS0'].


In [19]:
try:
    seg_aligns = get_alignment(prots_a)
    print(yaml.dump(seg_aligns, default_flow_style=False))
except:
    print("Doesnt work anymore...")

Searching for alignment: https://gpcrdb.org/services/alignment/protein/3SN6,7CFN,6E67,7CFM,7BZ2,7DHI,5G53,6NI3,7DHR,7D7M,6LI3,6GDG,7BW0/TM7/
Doesnt work anymore...


In [20]:
# Solution: manual download for the entire A class
al = load_alignment_()

In [21]:
al

,Unnamed: 0,TM7,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71
0,NaN,7.24x23,7.25x24,7.26x25,7.27x26,7.28x27,7.29x28,7.30x29,7.31x30,7.32x31,...,8.63x63,8.64x64,8.65x65,8.66x66,8.67x67,8.68x68,8.69x69,8.70x70,8.71x71,8.72x72
1,[Human] 5-HT1A receptor,_,_,_,_,_,_,_,P,T,...,_,_,_,_,_,_,_,_,_,_
2,[Human] 4iaq,_,_,_,_,_,_,_,H,L,...,_,_,_,_,_,_,_,_,_,_
3,[Human] 4iar,_,_,_,_,_,_,_,H,L,...,_,_,_,_,_,_,_,_,_,_
4,[Human] 5-HT1B receptor,_,_,_,_,_,_,_,H,L,...,_,_,_,_,_,_,_,_,_,_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,[Human] TAAR5,_,_,_,_,_,_,_,P,P,...,_,_,_,_,_,_,_,_,_,_
595,[Human] TAAR6,_,_,_,_,_,_,_,P,A,...,_,_,_,_,_,_,_,_,_,_
596,[Human] TAAR8,_,_,_,_,_,_,_,P,A,...,_,_,_,_,_,_,_,_,_,_
597,[Human] TAAR9,_,_,_,_,_,_,_,P,P,...,_,_,_,_,_,_,_,_,_,_


In [22]:
al_df = clean_alignment(al)

In [23]:
len(al_df)  # this is the entire class A!

598

In [24]:
# disp3(al_df)

In [25]:
# since the identifiers are only human readable...

In [26]:
genes = list(al_df['clean_id'])

l4=[]
l5=[]
l6=[]

for g in genes:
    if len(g) == 4:
        l4.append(g)
    elif len(g) == 5:
        l5.append(g)
    elif len(g) == 6:
        l6.append(g)
    else:
        pass

In [27]:
print(l4)  # these are (all?) pdbs

['4iaq', '4iar', '5v54', '6g79', '7c61', '6a93', '6a94', '6wgt', '6wh4', '6wha', '4ib4', '4nc3', '5tud', '5tvn', '6drx', '6dry', '6drz', '6ds0', '6bqg', '6bqh', '5cxv', '6oij', '6wjc', '3uon', '4mqs', '4mqt', '5yc8', '5zk3', '5zk8', '5zkb', '5zkc', '6oik', '6u1n', '5dsg', '6kp6', '6ol9', '6kux', '6kuy', '6k41', '6k42', '6kuw', '2r4r', '2r4s', '2rh1', '3d4s', '3kj6', '3ny8', '3ny9', '3nya', '3p0g', '3pds', '3sn6', '4gbr', '4lde', '4ldl', '4ldo', '4qkx', '5d5a', '5d5b', '5d6l', '5jqh', '5x7d', '6e67', '6mxt', '6n48', '6ni3', '6oba', '6prz', '6ps0', '6ps1', '6ps2', '6ps3', '6ps4', '6ps5', '6ps6', '7bz2', '7dhi', '7dhr', '6cm4', '6luq', '6vms', '7dfp', '3pbl', '5wiu', '5wiv', '3rze', '4yay', '4zud', '6do1', '6os0', '6os1', '6os2', '5unf', '5ung', '5unh', '5xjm', '6jod', '7c6a', '5vbl', '6knm', '5o9h', '6c1q', '6c1r', '5glh', '5gli', '5x93', '5xpr', '6igk', '6igl', '6k1q', '6lry', 'FPR1', '6lw5', '6omm', 'FPR3', '6ko5', '7br3', '6w25', '5zbh', '5zbq', '7ddz', '6os9', '6osa', '6pwc', '6up7',

In [28]:
# print(l5)  # these are?

In [29]:
# print(l6)  # another id..

In [30]:
# This should be put into a function somewhere or just left out!
inactive['pdb_lower'] = inactive.PDB.apply(lambda x: x.lower())
active['pdb_lower'] = active.PDB.apply(lambda x: x.lower())

<ipython-input-30-29d6b45c2935>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inactive['pdb_lower'] = inactive.PDB.apply(lambda x: x.lower())


In [31]:
found = []
for g in l4:
    select = inactive[inactive['pdb_lower'].str.contains(g.lower())]
    if len(select) > 0:
        found.append(g)

In [32]:
print("Found {} / {} inactive proteins.".format(len(found), len(inactive)))

Found 74 / 74 inactive proteins.


In [33]:
# check
# print(sorted(list(inactive['PDB'].str.lower()))[:10])
# print(sorted(l4)[:10])

In [34]:
structures_a = pdb_data(files_a)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  5.04it/s]


In [35]:
structures_i = pdb_data(files_i)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:12<00:00,  5.80it/s]


In [36]:
# disp3(structures_i) # missing segment information, segment position information, pp position information

In [37]:
# somehow the sequence lengths and polypeptide lengths are of unequal lengths... HOW?!
"""for i in range(50):
    print(structures_i['prot_len'].iloc[i], structures_i['pp_lens'].iloc[i])"""

"for i in range(50):\n    print(structures_i['prot_len'].iloc[i], structures_i['pp_lens'].iloc[i])"

In [38]:
full_a = structure_to_full(active, structures_a, al_df)

In [39]:
disp3(full_a)

,uniprot(gene),receptor family,Cl.,Species,Method,PDB,Refined Structure,Resolution,Preferred Chain,State,Degree active %,% of Seq1,Family,Subtype,Note,% of Seq2,Fusion,Antibodies,Name1,Type1,Function,Name2,Type2,D2x50 - S3x39,Sodium in structure,Authors,Reference,PDB date,Annotated,pdb_link,pdb_lower,ID,clean_id,TM7_combined,H8_combined,prot_id,prot_len,prot_seq,pp_ids,pp_lens,pp_seqs,psi_phi,full_prot_seq
0,GPR52,Aorphans,A(Rhodopsin),Human,cryo-EM,6LI3,6LI3_refined,3.3,R,Active,100,79.0,Gs,αs,Mini-Gs,61,-,Nanobody-35,Apo(noligand),none,Apo(noligand),-,-,Yes,No,"LinX,LiM,WangN,WuY,LuoZ,GuoS,HanG,LiS,YueY,Wei...",10.1038/S41586-020-2019-0,2020-02-26,Yes,https://files.rcsb.org/download/6LI3.pdb,6li3,[Human] 6li3,6li3,NPTLSFLTTWLAISNSFCNPVIYSLS,VFRLGLRRLSETMCT,6LI3,126,"[Q, V, Q, L, Q, E, S, G, G, G, L, V, Q, P, G, ...",[0],[126],"[(Q, V, Q, L, Q, E, S, G, G, G, L, V, Q, P, G,...","[[(None, 2.11539800963303), (-1.42884967419468...",QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQAPGKGLE...
1,GPBAR,Bileacid,A(Rhodopsin),Human,cryo-EM,7CFN,7CFN_refined,3.0,R,Active,100,83.0,Gs,αs,-,59,-,Nanobody-35,INT-777pubchemINT-777pubchem,smallmoleculesmallmolecule,AgonistPAM,"chenodeoxycholicacid,cholicacid,deoxycholicaci...",smallmolecule,No,No,"YangF,MaoC,GuoL,LinJ,MingQ,XiaoP,WuX,ShenQ,Guo...",10.1038/S41586-020-2569-1,2020-09-09,Yes,https://files.rcsb.org/download/7CFN.pdb,7cfn,[Human] 7cfn,7cfn,GPGTLLSLLSLGSASAAAVPVAMGLG,DQRYTA,7CFN,279,"[L, G, L, S, L, A, L, A, S, L, I, I, T, A, N, ...",[0],[274],"[(L, G, L, S, L, A, L, A, S, L, I, I, T, A, N,...","[[(None, -0.8080940478888483), (-1.18036277002...",LGLSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLL...
2,GPBAR,Bileacid,A(Rhodopsin),Human,cryo-EM,7CFM,7CFM_refined,3.0,R,Active,100,83.0,Gs,αs,-,58,-,Nanobody-35,P395pubchemchembl_ligand,none,Agonist,"chenodeoxycholicacid,cholicacid,deoxycholicaci...",smallmolecule,No,No,"YangF,MaoC,GuoL,LinJ,MingQ,XiaoP,WuX,ShenQ,Guo...",10.1038/S41586-020-2569-1,2020-09-09,Yes,https://files.rcsb.org/download/7CFM.pdb,7cfm,[Human] 7cfm,7cfm,LLSLLSLGSASAAAVPVAMGLG,,7CFM,278,"[L, S, L, A, L, A, S, L, I, I, T, A, N, L, L, ...",[0],[273],"[(L, S, L, A, L, A, S, L, I, I, T, A, N, L, L,...","[[(None, 1.2429597097836504), (-1.713221639440...",LSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLLTG...


In [40]:
full_a.to_csv('full_a.csv')
full_a = pd.read_csv('full_a.csv')

In [41]:
# full_a['full_prot_seq']

In [42]:
# full_a['TM7_combined']

In [43]:
# full_a['H8_combined']

In [44]:
full_a

,Unnamed: 0,uniprot(gene),receptor family,Cl.,Species,Method,PDB,Refined Structure,Resolution,Preferred Chain,...,TM7_combined,H8_combined,prot_id,prot_len,prot_seq,pp_ids,pp_lens,pp_seqs,psi_phi,full_prot_seq
0,0,GPR52,Aorphans,A(Rhodopsin),Human,cryo-EM,6LI3,6LI3_refined,3.3,R,...,NPTLSFLTTWLAISNSFCNPVIYSLS,VFRLGLRRLSETMCT,6LI3,126,"['Q', 'V', 'Q', 'L', 'Q', 'E', 'S', 'G', 'G', ...",[0],[126],[Seq('QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQA...,"[[(None, 2.11539800963303), (-1.42884967419468...",QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQAPGKGLE...
1,1,GPBAR,Bileacid,A(Rhodopsin),Human,cryo-EM,7CFN,7CFN_refined,3.0,R,...,GPGTLLSLLSLGSASAAAVPVAMGLG,DQRYTA,7CFN,279,"['L', 'G', 'L', 'S', 'L', 'A', 'L', 'A', 'S', ...",[0],[274],[Seq('LGLSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSL...,"[[(None, -0.8080940478888483), (-1.18036277002...",LGLSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLL...
2,2,GPBAR,Bileacid,A(Rhodopsin),Human,cryo-EM,7CFM,7CFM_refined,3.0,R,...,LLSLLSLGSASAAAVPVAMGLG,NaN,7CFM,278,"['L', 'S', 'L', 'A', 'L', 'A', 'S', 'L', 'I', ...",[0],[273],[Seq('LSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLL...,"[[(None, 1.2429597097836504), (-1.713221639440...",LSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLLTG...
3,3,AA2AR,Adenosine,A(Rhodopsin),Human,X-ray,5G53,5G53_refined,3.4,A,...,PLWLMYLAIVLSHTNSVVNPFIYAYR,NKEDLQTFRKIIRS,5G53,185,"['A', 'T', 'H', 'R', 'L', 'L', 'L', 'L', 'G', ...","[0, 1, 2]","[23, 17, 145]","[Seq('ATHRLLLLGADNSGKSTIVKQMR'), Seq('FETKFQVD...","[[(None, 1.0235583455244557), (-1.322712137199...",ATHRLLLLGADNSGKSTIVKQMRFETKFQVDKVNFHMFDVNDVTAI...
4,4,ADRB2,Adrenoceptors,A(Rhodopsin),Human,X-ray,3SN6,3SN6_refined,3.2,R,...,RKEVYILLNWIGYVNSGFNPLIYCR,SPDFRIAFQELLC,3SN6,128,"['Q', 'V', 'Q', 'L', 'Q', 'E', 'S', 'G', 'G', ...",[0],[128],[Seq('QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQA...,"[[(None, -2.901977459396043), (-2.405166956354...",QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQAPGKGLE...
5,5,GPBAR,Bileacid,A(Rhodopsin),Human,cryo-EM,7BW0,7BW0_refined,3.9,R,...,TLLSLLSLGSASAAAVPVAMG,NaN,7BW0,128,"['Q', 'V', 'Q', 'L', 'Q', 'E', 'S', 'G', 'G', ...",[0],[128],[Seq('QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQA...,"[[(None, 2.3251358383392553), (-1.742044222008...",QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQAPGKGLE...
6,6,AA2AR,Adenosine,A(Rhodopsin),Human,cryo-EM,6GDG,6GDG_refined,4.1,A,...,PLWLMYLAIVLSHTNSVVNPFIYAYR,IREFRQTF,6GDG,128,"['Q', 'V', 'Q', 'L', 'Q', 'E', 'S', 'G', 'G', ...",[0],[128],[Seq('QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQA...,"[[(None, 1.1181014427736993), (0.6973683110658...",QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQAPGKGLE...
7,7,ADRB2,Adrenoceptors,A(Rhodopsin),Human,X-ray,6E67,6E67_refined,3.7,B,...,RKEVYILLNWIGYVNSGFNP,SPDFRIAFQELL,6E67,477,"['F', 'L', 'L', 'A', 'P', 'N', 'R', 'S', 'H', ...",[0],[476],[Seq('FLLAPNRSHAPDHDVTQQRDEVWVVGMGIVMSLIVLAIVF...,"[[(None, 0.4238732643408313), (-1.326951794170...",FLLAPNRSHAPDHDVTQQRDEVWVVGMGIVMSLIVLAIVFGNVLVI...
8,8,ADRB2,Adrenoceptors,A(Rhodopsin),Human,cryo-EM,7BZ2,7BZ2_refined,3.8,R,...,RKEVYILLNWIGYVNSGFNPLIY,SPDFRIAFQ,7BZ2,286,"['E', 'V', 'W', 'V', 'V', 'G', 'M', 'G', 'I', ...","[0, 1, 2]","[148, 61, 76]",[Seq('EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTN...,"[[(None, 2.3630642845093455), (-2.179417972707...",EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSL...
9,9,ADRB2,Adrenoceptors,A(Rhodopsin),Human,cryo-EM,7DHR,7DHR_refined,3.8,R,...,ILLNWIGYVNSGFNPLIY,SPDFRIAFQEL,7DHR,285,"['E', 'V', 'W', 'V', 'V', 'G', 'M', 'G', 'I', ...","[0, 1, 2]","[147, 63, 74]",[Seq('EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTN...,"[[(None, -1.254930729396107), (-1.858914773043...",EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSL...


In [45]:
complete_a = complete_data(full_a, target='NPFIY')

In [46]:
print(len(full_a))
print(len(complete_a))

13
13


In [47]:
disp3(complete_a)

,Unnamed: 0,uniprot(gene),receptor family,Cl.,Species,Method,PDB,Refined Structure,Resolution,Preferred Chain,State,Degree active %,% of Seq1,Family,Subtype,Note,% of Seq2,Fusion,Antibodies,Name1,Type1,Function,Name2,Type2,D2x50 - S3x39,Sodium in structure,Authors,Reference,PDB date,Annotated,pdb_link,pdb_lower,ID,clean_id,TM7_combined,H8_combined,prot_id,prot_len,prot_seq,pp_ids,pp_lens,pp_seqs,psi_phi,full_prot_seq,start,end,mean,std,score,res,target_wrt_tm7
0,0,GPR52,Aorphans,A(Rhodopsin),Human,cryo-EM,6LI3,6LI3_refined,3.3,R,Active,100,79.0,Gs,αs,Mini-Gs,61,-,Nanobody-35,Apo(noligand),none,Apo(noligand),-,-,Yes,No,"LinX,LiM,WangN,WuY,LuoZ,GuoS,HanG,LiS,YueY,Wei...",10.1038/S41586-020-2019-0,2020-02-26,Yes,https://files.rcsb.org/download/6LI3.pdb,6li3,[Human] 6li3,6li3,PGGSLRLSCAASGFTFSNYKMNWVRQAPGKGLEWVSDISQSGASIS...,VFRLGLRRLSETMCT,6LI3,126,"['Q', 'V', 'Q', 'L', 'Q', 'E', 'S', 'G', 'G', ...",[0],[126],[Seq('QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQA...,"[[(None, 2.11539800963303), (-1.42884967419468...",QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQAPGKGLE...,13,126,63.346154,36.219356,1.507692,"[13, 14, 20, 21, 26, 28, 29, 30, 38, 39, 48, 5...","(21, 109, 52.2, 29.006206232459977, 0.84000000..."
1,1,GPBAR,Bileacid,A(Rhodopsin),Human,cryo-EM,7CFN,7CFN_refined,3.0,R,Active,100,83.0,Gs,αs,-,59,-,Nanobody-35,INT-777pubchemINT-777pubchem,smallmoleculesmallmolecule,AgonistPAM,"chenodeoxycholicacid,cholicacid,deoxycholicaci...",smallmolecule,No,No,"YangF,MaoC,GuoL,LinJ,MingQ,XiaoP,WuX,ShenQ,Guo...",10.1038/S41586-020-2569-1,2020-09-09,Yes,https://files.rcsb.org/download/7CFN.pdb,7cfn,[Human] 7cfn,7cfn,GPGTLLSLLSLGSASAAAVPVAMGLGDQRYT,DQRYTA,7CFN,279,"['L', 'G', 'L', 'S', 'L', 'A', 'L', 'A', 'S', ...",[0],[274],[Seq('LGLSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSL...,"[[(None, -0.8080940478888483), (-1.18036277002...",LGLSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLL...,240,271,252.500000,7.500000,2.046154,"[240, 241, 242, 243, 244, 245, 246, 247, 248, ...","(19, 31, 26.4, 5.6780278266313555, 0.56, [19, ..."
2,2,GPBAR,Bileacid,A(Rhodopsin),Human,cryo-EM,7CFM,7CFM_refined,3.0,R,Active,100,83.0,Gs,αs,-,58,-,Nanobody-35,P395pubchemchembl_ligand,none,Agonist,"chenodeoxycholicacid,cholicacid,deoxycholicaci...",smallmolecule,No,No,"YangF,MaoC,GuoL,LinJ,MingQ,XiaoP,WuX,ShenQ,Guo...",10.1038/S41586-020-2569-1,2020-09-09,Yes,https://files.rcsb.org/download/7CFM.pdb,7cfm,[Human] 7cfm,7cfm,LLSLLSLGSASAAAVPVAMGLGDQRYT,NaN,7CFM,278,"['L', 'S', 'L', 'A', 'L', 'A', 'S', 'L', 'I', ...",[0],[273],[Seq('LSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLL...,"[[(None, 1.2429597097836504), (-1.713221639440...",LSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLLTG...,242,269,252.500000,6.344289,1.859091,"[242, 243, 244, 245, 246, 247, 248, 249, 250, ...","(15, 27, 22.4, 5.6780278266313555, 0.639999999..."


In [48]:
# Filter out poorly aligned proteins
# disp3(complete_a[complete_a['std'] < 20])

In [49]:
#complete_i['pp_lens']

In [50]:
al_df

,ID,clean_id,TM7_combined,H8_combined
1,[Human] 5-HT1A receptor,5-HT1A receptor,PTLLGAIINWLGYSNSLLNPVIYAYF,NKDFQNAFKKII
2,[Human] 4iaq,4iaq,HLAIFDFFTWLGYLNSLINPIIYTMS,NEDFKQAFHKLI
3,[Human] 4iar,4iar,HLAIFDFFTWLGYLNSLINPIIYTM,NEDFKQAFHKLI
4,[Human] 5-HT1B receptor,5-HT1B receptor,HLAIFDFFTWLGYLNSLINPIIYTMS,NEDFKQAFHKLI
5,[Human] 5v54,5v54,HLAIFDFFTWLGYLNSLINPIIYTMS,NEDFKQAFHKLI
...,...,...,...,...
594,[Human] TAAR5,TAAR5,PPLVFDIFIWFAYFNSACNPIIYVFS,YQWFRKALKLTL
595,[Human] TAAR6,TAAR6,PACIYEICCWCAYYNSAMNPLIYALF,YPWFRKAIKVIV
596,[Human] TAAR8,TAAR8,PAYIYEICCWSAYYNSAMNPLIYALF,YPWFRKAIKLIL
597,[Human] TAAR9,TAAR9,PPYVYEILVWCVYYNSAMNPLIYAFF,YQWFGKAIKLIV


In [4]:
full_i = structure_to_full(inactive, structures_i, al_df)

NameError: name 'inactive' is not defined

In [4]:
full_i.to_csv('full_i.csv')

NameError: name 'full_i' is not defined

In [4]:
full_i = pd.read_csv('full_i.csv')

In [5]:
full_a = pd.read_csv('full_a.csv')

In [6]:
print(len(full_i))

73


In [7]:
complete_i = complete_data(full_i, max_std_alignment=None, elongate=True, padding_r=5, target='NPFIY', filter_bad_checks=True)
# this is the part where we want to increase the target range (since it is inconsistent)

In [11]:
disp3(complete_i[['TM7_combined', 'TM7_found']])

,TM7_combined,TM7_found
5,KEVYILLNWIGYVNSGFNPLIYCR,KEVYILLNWIGYVNSGFNPLIYCRSPDFRIAFQELLCL
6,KEVYILLNWIGYVNSGFNPLIYCR,KEVYILLNWIGYVNSGFNPLIYCRSPDFRIAFQELLCL
7,KEVYILLNWIGYVNSGFNPLIYCR,KEVYILLNWIGYVNSGFNPLIYCRSPDFRIAFQELLCL


In [8]:
complete_a = complete_data(full_a, max_std_alignment=None, elongate=True, padding_r=5, target='NPFIY', filter_bad_checks=True)
# this is the part where we want to increase the target range (since it is inconsistent)

In [12]:
disp3(complete_a[['TM7_combined', 'TM7_found']])

,TM7_combined,TM7_found
1,GPGTLLSLLSLGSASAAAVPVAMGLG,GPGTLLSLLSLGSASAAAVPVAMGLGDQRYTAPW
2,LLSLLSLGSASAAAVPVAMGLG,LLSLLSLGSASAAAVPVAMGLGDQRYTAPWR
8,RKEVYILLNWIGYVNSGFNPLIY,RKEVYILLNWIGYVNSGFNPLIYSRSPDFRIAF


In [ ]:
# disp3(complete_i['start'])

In [ ]:
# len(complete_i[complete_i['std'] < 10])

In [ ]:
len(complete_i)

In [ ]:
# Fix that number ~ we have 73 proteins in the full datatable but end up with 87 
# after the segments alignment is complete (unfiltered)
# disp3(complete_i['TM7_combined'])

In [ ]:
disp3(complete_i[['PDB', 'TM7_combined', 'target_wrt_tm7']])

In [33]:
target = 'NPLIY'

In [34]:
def get_target_df(complete: pd.DataFrame, target='NPXXY', valid=True):
    def sum_start(a, b):
        return a + b[0]
    def sum_end(a, b):
        return a + b[1]
    def extend(ls):
        is_list = isinstance(ls[0], list)
        for i, l in enumerate(ls):
            if is_list & i == 0:
                out = l
            elif is_list:
                out.extend(l)
            else:
                return ls
        return out
    def clean_pps(x):
        return ast.literal_eval(x.replace('Seq', '').replace('(', '').replace(')', ''))
    target_df = complete[['PDB', 'uniprot(gene)', 'Resolution', 'PDB date', \
                          'TM7_found', 'score', 'prot_len', 'full_prot_seq', 'start']].copy()
    target_df['pp_seqs'] = complete.apply(lambda x: clean_pps(x.pp_seqs), axis=1).copy()
    target_df['pp_seq_lens'] = target_df.apply(lambda x: [len(y) for y in x.pp_seqs], axis=1).copy()
    # target_df.loc[:, 'target_seq'] = target
    target_df['start_absolute'] = complete.apply(lambda x: sum_start(x.start, x.target_wrt_tm7), axis=1).copy()
    target_df['end_absolute'] = complete.apply(lambda x: sum_end(x.start, x.target_wrt_tm7), axis=1).copy()
    target_df['start'] = complete['target_wrt_tm7'].apply(lambda x: x[0]).copy()
    target_df['end'] = complete['target_wrt_tm7'].apply(lambda x: x[1]).copy()
    target_df['full_aligned_seg'] = target_df.apply(lambda x: x.full_prot_seq[x.start_absolute:x.end_absolute], axis=1)
    target_df['TM7_aligned_seg'] = target_df.apply(lambda x: x.TM7_found[x.start:x.end], axis=1)
    target_df['psi_phi'] = complete.apply(lambda x: extend(ast.literal_eval(x.psi_phi)), axis=1)
    if valid:
        target_df['target_angles'] = target_df.apply(lambda x: x.psi_phi[x.start_absolute:x.end_absolute], axis=1)
        target_df = target_df[target_df['full_aligned_seg']==target_df['TM7_aligned_seg']]
        return target_df  # [len(target_df['target_angles']) >= len(target)]
    else:
        return target_df

In [35]:
valid_a = get_target_df(complete_a, target, valid=True)

In [37]:
valid_a

,PDB,uniprot(gene),Resolution,PDB date,TM7_found,score,prot_len,full_prot_seq,start,pp_seqs,pp_seq_lens,start_absolute,end_absolute,end,full_aligned_seg,TM7_aligned_seg,psi_phi,target_angles
1,7CFN,GPBAR,3.0,2020-09-09,GPGTLLSLLSLGSASAAAVPVAMGLGDQRYTAPW,2.046154,279,LGLSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLL...,19,[LGLSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGL...,[60],259,266,26,PVAMGLG,PVAMGLG,"[(None, -0.8080940478888483), (-1.180362770021...","[(-1.1727213988088117, -0.5301388085987634), (..."
2,7CFM,GPBAR,3.0,2020-09-09,LLSLLSLGSASAAAVPVAMGLGDQRYTAPWR,1.859091,278,LSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLLTG...,15,[LSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLLT...,[60],257,264,22,PVAMGLG,PVAMGLG,"[(None, 1.2429597097836504), (-1.7132216394400...","[(-1.2096660283557674, -0.5027183977621975), (..."
8,7BZ2,ADRB2,3.8,2020-08-05,RKEVYILLNWIGYVNSGFNPLIYSRSPDFRIAF,1.860870,286,EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSL...,18,[EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITS...,"[60, 60, 60]",266,271,23,NPLIY,NPLIY,"[(None, 0.7247878775593547), (-1.3430951956042...",[]
9,7DHR,ADRB2,3.8,2020-12-16,ILLNWIGYVNSGFNPLIYSRSPDFRIAF,1.522222,285,EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSL...,13,[EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITS...,"[60, 60, 60]",266,271,18,NPLIY,NPLIY,"[(None, 2.548423928842914), (-1.54965132845452...",[]
10,7DHI,ADRB2,3.3,2020-12-16,EVYILLNWIGYVNSGFNPLIYSRSPDFRIAF,1.719048,291,EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSL...,16,[EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITS...,"[60, 60]",270,275,21,NPLIY,NPLIY,"[(None, 2.220761123286891), (-2.16442545459555...","[(-1.1948225616813992, -0.5293781576224712), (..."


In [38]:
valid_a.to_csv('valid_a.csv', index=False)

In [39]:
valid_i = get_target_df(complete_i, target, valid=True)
# Filter for proteins where the alignments match ~ indicates that angles are valid and the region is present
# if only valids we compare if the full aligned segment withthe TM7 aligned segment
valid_i.to_csv('valid_i.csv', index=False)

In [19]:
disp3(valid_a)

,PDB,uniprot(gene),Resolution,PDB date,TM7_found,score,prot_len,full_prot_seq,start,pp_seqs,pp_seq_lens,start_absolute,end_absolute,end,full_aligned_seg,TM7_aligned_seg,psi_phi,target_angles
1,7CFN,GPBAR,3.0,2020-09-09,GPGTLLSLLSLGSASAAAVPVAMGLGDQRYTAPW,2.046154,279,LGLSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLL...,19,[LGLSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGL...,[60],259,266,26,PVAMGLG,PVAMGLG,"[(None, -0.8080940478888483), (-1.180362770021...","[(-1.1727213988088117, -0.5301388085987634), (..."
2,7CFM,GPBAR,3.0,2020-09-09,LLSLLSLGSASAAAVPVAMGLGDQRYTAPWR,1.859091,278,LSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLLTG...,15,[LSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLLT...,[60],257,264,22,PVAMGLG,PVAMGLG,"[(None, 1.2429597097836504), (-1.7132216394400...","[(-1.2096660283557674, -0.5027183977621975), (..."
8,7BZ2,ADRB2,3.8,2020-08-05,RKEVYILLNWIGYVNSGFNPLIYSRSPDFRIAF,1.860870,286,EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSL...,18,[EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITS...,"[60, 60, 60]",266,271,23,NPLIY,NPLIY,"[(None, 0.7247878775593547), (-1.3430951956042...",[]


In [20]:
disp3(valid_i)

,PDB,uniprot(gene),Resolution,PDB date,TM7_found,score,prot_len,full_prot_seq,start,pp_seqs,pp_seq_lens,start_absolute,end_absolute,end,full_aligned_seg,TM7_aligned_seg,psi_phi,target_angles
5,6PS1,ADRB2,3.2,2019-11-13,KEVYILLNWIGYVNSGFNPLIYCRSPDFRIAFQELLCL,1.258333,450,DEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITS...,17,[DEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFIT...,[60],421,428,24,NPLIYCR,NPLIYCR,"[(None, 2.5256872329638655), (-1.0111750694833...","[(-0.8380310981643087, -0.8003636915192417), (..."
6,6PS6,ADRB2,2.7,2019-11-13,KEVYILLNWIGYVNSGFNPLIYCRSPDFRIAFQELLCL,1.254167,470,RDEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFIT...,17,[RDEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFI...,[60],422,429,24,NPLIYCR,NPLIYCR,"[(None, 0.27689763066035566), (-0.793226952381...","[(-0.8287561614163858, -0.8590474805592928), (..."
7,6PS5,ADRB2,2.9,2019-11-13,KEVYILLNWIGYVNSGFNPLIYCRSPDFRIAFQELLCL,1.258333,454,DEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITS...,17,[DEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFIT...,[60],421,428,24,NPLIYCR,NPLIYCR,"[(None, 2.9306733892826236), (-1.0267900603457...","[(-0.8621097350371153, -0.8237030852743529), (..."


In [21]:
valid_a.target_angles

1     [(-1.1727213988088117, -0.5301388085987634), (...
2     [(-1.2096660283557674, -0.5027183977621975), (...
8                                                    []
9                                                    []
10    [(-1.1948225616813992, -0.5293781576224712), (...
Name: target_angles, dtype: object

In [22]:
valid_a

,PDB,uniprot(gene),Resolution,PDB date,TM7_found,score,prot_len,full_prot_seq,start,pp_seqs,pp_seq_lens,start_absolute,end_absolute,end,full_aligned_seg,TM7_aligned_seg,psi_phi,target_angles
1,7CFN,GPBAR,3.0,2020-09-09,GPGTLLSLLSLGSASAAAVPVAMGLGDQRYTAPW,2.046154,279,LGLSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLL...,19,[LGLSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGL...,[60],259,266,26,PVAMGLG,PVAMGLG,"[(None, -0.8080940478888483), (-1.180362770021...","[(-1.1727213988088117, -0.5301388085987634), (..."
2,7CFM,GPBAR,3.0,2020-09-09,LLSLLSLGSASAAAVPVAMGLGDQRYTAPWR,1.859091,278,LSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLLTG...,15,[LSLALASLIITANLLLALGIAWDRRLRSPPAGCFFLSLLLAGLLT...,[60],257,264,22,PVAMGLG,PVAMGLG,"[(None, 1.2429597097836504), (-1.7132216394400...","[(-1.2096660283557674, -0.5027183977621975), (..."
8,7BZ2,ADRB2,3.8,2020-08-05,RKEVYILLNWIGYVNSGFNPLIYSRSPDFRIAF,1.860870,286,EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSL...,18,[EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITS...,"[60, 60, 60]",266,271,23,NPLIY,NPLIY,"[(None, 0.7247878775593547), (-1.3430951956042...",[]
9,7DHR,ADRB2,3.8,2020-12-16,ILLNWIGYVNSGFNPLIYSRSPDFRIAF,1.522222,285,EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSL...,13,[EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITS...,"[60, 60, 60]",266,271,18,NPLIY,NPLIY,"[(None, 2.548423928842914), (-1.54965132845452...",[]
10,7DHI,ADRB2,3.3,2020-12-16,EVYILLNWIGYVNSGFNPLIYSRSPDFRIAF,1.719048,291,EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSL...,16,[EVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITS...,"[60, 60]",270,275,21,NPLIY,NPLIY,"[(None, 2.220761123286891), (-2.16442545459555...","[(-1.1948225616813992, -0.5293781576224712), (..."


In [23]:
np.asarray(valid_i['target_angles'].iloc[0])

array([[-0.8380311 , -0.80036369],
       [-1.10427995, -0.60851755],
       [-1.21469441, -0.48920851],
       [-1.36171012, -0.44312874],
       [-1.24633608, -0.13742096],
       [-1.19782354, -0.27717333],
       [-1.22382635, -0.68829526]])

In [24]:
np.asarray(valid_a['target_angles'].iloc[0])

array([[-1.1727214 , -0.53013881],
       [-1.22515482, -0.52101624],
       [-1.32065849, -0.51057197],
       [-1.30211179, -0.57180035],
       [-1.22680805, -0.83150684],
       [-1.23505798, -0.23439765],
       [-1.13071709, -0.99517171]])

In [25]:
valid_i.to_csv('valid_i.csv')

In [26]:
# j, o, u and x do not code for an amino acid (sometimes used to represent ?)

In [27]:
len(valid_i[valid_i['full_aligned_seg']==valid_i['TM7_aligned_seg']])

42

In [28]:
x = valid_i[['prot_len', 'psi_phi']]

In [29]:
x['len_psi_phi'] = valid_i.apply(lambda x: len(x.psi_phi), axis=1)

<ipython-input-29-0d223a10ad61>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['len_psi_phi'] = valid_i.apply(lambda x: len(x.psi_phi), axis=1)


In [30]:
x

,prot_len,psi_phi,len_psi_phi
5,450,"[(None, 2.5256872329638655), (-1.0111750694833...",442
6,470,"[(None, 0.27689763066035566), (-0.793226952381...",443
7,454,"[(None, 2.9306733892826236), (-1.0267900603457...",442
8,452,"[(None, -0.46658462888578983), (-1.13290162531...",440
10,471,"[(None, 2.747201390727765), (-1.45410174600076...",442
11,499,"[(None, -2.5985189892267564), (-1.490179980824...",443
12,449,"[(None, -0.6015786016853899), (-1.196694536887...",440
13,447,"[(None, 1.8948701581079868), (-1.1309253657103...",390
14,539,"[(None, 1.6380759814786037), (-1.0785675902132...",387
15,532,"[(None, 2.003678106751159), (-1.08420177868628...",388


In [ ]:
# match each of the active complexes to all the inactive ones (based on the uniprot genes)

In [ ]:
# interaction dataframe

interaction_df = pd.DataFrame(columns=['PDB_active', 'PDB_inactive', 'phi_psi_active', 'phi_psi_inactive', 'delta_phi_psi'])
# this should be done here...